### IMPORTING DEPENDENCIES

In [1]:
import pandas as pd
import numpy as np
import sqlite3

### READING CRIME DATA CSVS AND TRANSFORMATION

In [2]:
raw_df_19_20 = pd.read_csv("2019-20-fullyr-data_sa_crime.csv") # returns 95702 entries  C:\Users\Laura\Desktop\Project-2-Visualisations\Data\raw_data\2019-20-fullyr-data_sa_crime.csv
raw_df_18_19 = pd.read_csv("2018-19-data_sa_crime.csv")        # returns 94937 entries
raw_df_17_18 = pd.read_csv("2017-18-data_sa_crime.csv")        # returns 90655 entries
raw_df_16_17 = pd.read_csv("2016-17-data_sa_crime.csv")        # returns 90682 entries
raw_df_15_16 = pd.read_csv("2015-16-data_sa_crime.csv")        # returns 93167 entries
raw_df_14_15 = pd.read_csv("2014-15-data_sa_crime.csv")        # returns 93167 entries

In [3]:
# combining dataframes
frames = [raw_df_14_15, raw_df_15_16 , raw_df_16_17 , raw_df_17_18 , raw_df_18_19 , raw_df_19_20]
raw_combined_crime = pd.concat(frames)

In [4]:
# look for missing values
raw_combined_crime.count()

Reported Date                  555414
Suburb - Incident              554240
Postcode - Incident            553205
Offence Level 1 Description    555414
Offence Level 2 Description    555414
Offence Level 3 Description    555414
Offence count                  555414
dtype: int64

In [5]:
# drop missing values
clean_crime = raw_combined_crime.dropna(how='any')

In [6]:
# rename cols
clean_crime = clean_crime.rename(columns={
    "Reported Date": "date",
    "Suburb - Incident":"locality",
    "Postcode - Incident": "postcode",
    "Offence Level 1 Description": "offence1",
    "Offence Level 2 Description": "offence2",
    "Offence Level 3 Description": "offence3",
    "Offence count": "count",})

In [7]:
# check for null values
clean_crime.isnull().sum()

date        0
locality    0
postcode    0
offence1    0
offence2    0
offence3    0
count       0
dtype: int64

In [8]:
# check for duplicates
clean_crime.duplicated().sum()

0

In [9]:
clean_crime.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 553181 entries, 0 to 95699
Data columns (total 7 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   date      553181 non-null  object 
 1   locality  553181 non-null  object 
 2   postcode  553181 non-null  object 
 3   offence1  553181 non-null  object 
 4   offence2  553181 non-null  object 
 5   offence3  553181 non-null  object 
 6   count     553181 non-null  float64
dtypes: float64(1), object(6)
memory usage: 33.8+ MB


### READING POSTCODES CSV AND TRANSFORMATION FOR GEO INFORMATION

In [10]:
postcodes_df = pd.read_csv("australian_postcodes.csv")
postcodes_df.head()

,id,postcode,locality,state,long,lat,dc,type,status,sa3,...,SA3_NAME_2016,SA4_CODE_2016,SA4_NAME_2016,RA_2011,RA_2016,MMM_2015,MMM_2019,ced,altitude,chargezone
0,230,200,ANU,ACT,149.119000,-35.277700,NaN,NaN,NaN,NaN,...,North Canberra,801.0,Australian Capital Territory,1.0,1.0,1.0,1.0,NaN,NaN,N2
1,21820,200,Australian National University,ACT,149.118900,-35.277700,NaN,NaN,Added 19-Jan-2020,NaN,...,North Canberra,801.0,Australian Capital Territory,1.0,1.0,1.0,1.0,NaN,NaN,N2
2,232,800,DARWIN,NT,130.836680,-12.458684,NaN,NaN,Updated 6-Feb-2020,70101.0,...,Darwin City,701.0,Darwin,3.0,3.0,2.0,2.0,NaN,NaN,NT1
3,233,801,DARWIN,NT,130.836680,-12.458684,NaN,NaN,Updated 25-Mar-2020 SA3,70101.0,...,Darwin City,701.0,Darwin,3.0,3.0,2.0,2.0,NaN,NaN,NT1
4,234,804,PARAP,NT,130.873315,-12.428017,NaN,NaN,Updated 25-Mar-2020 SA3,70102.0,...,Darwin City,701.0,Darwin,3.0,3.0,2.0,2.0,NaN,NaN,NT1


In [11]:
sa_postcodes = postcodes_df[postcodes_df["state"]=="SA"]
sa_geo_info = sa_postcodes[["locality", "lat","long"]]

In [12]:
sa_geo_info

,locality,lat,long
305,AMATA,-26.127662,131.173880
308,ANANGU PITJANTJATJARA YANKUNYTJATJARA,-27.077283,131.205697
314,AYERS RANGE SOUTH,-26.190601,133.268032
322,DE ROSE HILL,-26.556097,133.311196
326,ERNABELLA,-21.949513,131.298809
...,...,...,...
15671,ADELAIDE,-35.120097,139.273782
15672,ADELAIDE,-35.120097,139.273782
15673,REGENCY PARK,-34.860017,138.565906
15674,ADELAIDE AIRPORT,-34.945146,138.530183


In [18]:
# dropping any duplicates for locality
sa_geo_info_cleaned = sa_geo_info.drop_duplicates(subset="locality")

In [19]:
sa_geo_info_cleaned

,locality,lat,long
305,AMATA,-26.127662,131.173880
308,ANANGU PITJANTJATJARA YANKUNYTJATJARA,-27.077283,131.205697
314,AYERS RANGE SOUTH,-26.190601,133.268032
322,DE ROSE HILL,-26.556097,133.311196
326,ERNABELLA,-21.949513,131.298809
...,...,...,...
15667,SIMPSON DESERT,-26.678030,137.432565
15668,TODMORDEN,-27.283873,134.827062
15669,WITJIRA,-26.338510,135.675034
15674,ADELAIDE AIRPORT,-34.945146,138.530183


In [22]:
# merging crime data with geo data
df = pd.merge(clean_crime, sa_geo_info_cleaned, on="locality", how="left")

In [23]:
df.head()

,date,locality,postcode,offence1,offence2,offence3,count,lat,long
0,1/07/2014,ADELAIDE,5000,OFFENCES AGAINST PROPERTY,FRAUD DECEPTION AND RELATED OFFENCES,Obtain benefit by deception,2.0,-34.937459,138.608637
1,1/07/2014,ADELAIDE,5000,OFFENCES AGAINST PROPERTY,FRAUD DECEPTION AND RELATED OFFENCES,"Other fraud, deception and related offences",1.0,-34.937459,138.608637
2,1/07/2014,ADELAIDE,5000,OFFENCES AGAINST PROPERTY,PROPERTY DAMAGE AND ENVIRONMENTAL,Graffiti,1.0,-34.937459,138.608637
3,1/07/2014,ADELAIDE,5000,OFFENCES AGAINST PROPERTY,PROPERTY DAMAGE AND ENVIRONMENTAL,Other property damage and environmental,2.0,-34.937459,138.608637
4,1/07/2014,ADELAIDE,5000,OFFENCES AGAINST PROPERTY,SERIOUS CRIMINAL TRESPASS,Other unlawful entry with intent,7.0,-34.937459,138.608637


In [24]:
# converting date col to date format
df["date"] = pd.to_datetime(df["date"])


### CREATING TABLES FOR VISUALISATIONS

##### MAIN DF

In [25]:
df.head()

,date,locality,postcode,offence1,offence2,offence3,count,lat,long
0,2014-01-07,ADELAIDE,5000,OFFENCES AGAINST PROPERTY,FRAUD DECEPTION AND RELATED OFFENCES,Obtain benefit by deception,2.0,-34.937459,138.608637
1,2014-01-07,ADELAIDE,5000,OFFENCES AGAINST PROPERTY,FRAUD DECEPTION AND RELATED OFFENCES,"Other fraud, deception and related offences",1.0,-34.937459,138.608637
2,2014-01-07,ADELAIDE,5000,OFFENCES AGAINST PROPERTY,PROPERTY DAMAGE AND ENVIRONMENTAL,Graffiti,1.0,-34.937459,138.608637
3,2014-01-07,ADELAIDE,5000,OFFENCES AGAINST PROPERTY,PROPERTY DAMAGE AND ENVIRONMENTAL,Other property damage and environmental,2.0,-34.937459,138.608637
4,2014-01-07,ADELAIDE,5000,OFFENCES AGAINST PROPERTY,SERIOUS CRIMINAL TRESPASS,Other unlawful entry with intent,7.0,-34.937459,138.608637


##### LEVEL 1 OFFENCE: BY YEAR

In [26]:
lev1_2014 = df[(df["date"] >= "2014-01-01") & (df["date"] <= "2014-12-31")].groupby("offence1")["count"].sum()
lev1_2015 =  df[(df["date"] >= "2015-01-01") & (df["date"] <= "2015-12-31")].groupby("offence1")["count"].sum()
lev1_2016 =  df[(df["date"] >= "2016-01-01") & (df["date"] <= "2016-12-31")].groupby("offence1")["count"].sum()
lev1_2017 =  df[(df["date"] >= "2017-01-01") & (df["date"] <= "2017-12-31")].groupby("offence1")["count"].sum()
lev1_2018 =  df[(df["date"] >= "2018-01-01") & (df["date"] <= "2018-12-31")].groupby("offence1")["count"].sum()
lev1_2019 =  df[(df["date"] >= "2019-01-01") & (df["date"] <= "2019-12-31")].groupby("offence1")["count"].sum()
lev1_2020 =  df[(df["date"] >= "2020-01-01") & (df["date"] <= "2020-12-31")].groupby("offence1")["count"].sum()
lev1_all_years = df.groupby("offence1")["count"].sum()

In [29]:
lev1_2014["YEAR"]="2014"
lev1_2014 = pd.DataFrame(data=lev1_2014)
lev1_2014=lev1_2014.T
lev1_2014.set_index("YEAR")

offence1,OFFENCES AGAINST PROPERTY,OFFENCES AGAINST THE PERSON
YEAR,,
2014,39893.0,12336.0


In [30]:
lev1_2015["YEAR"]="2015"
lev1_2015 = pd.DataFrame(data=lev1_2015)
lev1_2015=lev1_2015.T
lev1_2015.set_index("YEAR")

offence1,OFFENCES AGAINST PROPERTY,OFFENCES AGAINST THE PERSON
YEAR,,
2015,82978.0,24320.0


In [31]:
lev1_2016["YEAR"]="2016"
lev1_2016 = pd.DataFrame(data=lev1_2016)
lev1_2016=lev1_2016.T
lev1_2016.set_index("YEAR")

offence1,OFFENCES AGAINST PROPERTY,OFFENCES AGAINST THE PERSON
YEAR,,
2016,86440.0,23124.0


In [36]:
lev1_2017["YEAR"]="2017"
lev1_2017 = pd.DataFrame(data=lev1_2017)
lev1_2017=lev1_2017.T
lev1_2017.set_index("YEAR")

offence1,OFFENCES AGAINST PROPERTY,OFFENCES AGAINST THE PERSON
YEAR,,
2017,80987.0,22235.0


In [32]:
lev1_2018["YEAR"]="2018"
lev1_2018 = pd.DataFrame(data=lev1_2018)
lev1_2018=lev1_2018.T
lev1_2018.set_index("YEAR")

offence1,OFFENCES AGAINST PROPERTY,OFFENCES AGAINST THE PERSON
YEAR,,
2018,87951.0,22150.0


In [35]:
lev1_2019["YEAR"]="2019"
lev1_2019 = pd.DataFrame(data=lev1_2019)
lev1_2019=lev1_2019.T
lev1_2019.set_index("YEAR")

offence1,OFFENCES AGAINST PROPERTY,OFFENCES AGAINST THE PERSON
YEAR,,
2019,96932.0,20910.0


In [33]:
lev1_2020["YEAR"]="2020"
lev1_2020 = pd.DataFrame(data=lev1_2020)
lev1_2020=lev1_2020.T
lev1_2020.set_index("YEAR")

offence1,OFFENCES AGAINST PROPERTY,OFFENCES AGAINST THE PERSON
YEAR,,
2020,42987.0,10531.0


In [38]:
lev1_all_years["YEAR"]="ALL-YEARS"
lev1_all_years = pd.DataFrame(data=lev1_all_years)
lev1_all_years=lev1_all_years.T
lev1_all_years.set_index("YEAR")

offence1,OFFENCES AGAINST PROPERTY,OFFENCES AGAINST THE PERSON
YEAR,,
ALL-YEARS,518168.0,135606.0
ALL-YEARS,ALL-YEARS,ALL-YEARS


In [39]:
frames = [lev1_2014, lev1_2015 , lev1_2016 , lev1_2017 , lev1_2018 , lev1_2019, lev1_2020, lev1_all_years]
lev1_combined = pd.concat(frames)

In [40]:
lev1_combined = lev1_combined.set_index("YEAR")

##### LEVEL 2 OFFENCE: BY YEAR

In [42]:
lev2_2014 =  df[(df["date"] >= "2014-01-01") & (df["date"] <= "2014-12-31")].groupby("offence2")["count"].sum()
lev2_2015 =  df[(df["date"] >= "2015-01-01") & (df["date"] <= "2015-12-31")].groupby("offence2")["count"].sum()
lev2_2016 =  df[(df["date"] >= "2016-01-01") & (df["date"] <= "2016-12-31")].groupby("offence2")["count"].sum()
lev2_2017 =  df[(df["date"] >= "2017-01-01") & (df["date"] <= "2017-12-31")].groupby("offence2")["count"].sum()
lev2_2018 =  df[(df["date"] >= "2018-01-01") & (df["date"] <= "2018-12-31")].groupby("offence2")["count"].sum()
lev2_2019 =  df[(df["date"] >= "2019-01-01") & (df["date"] <= "2019-12-31")].groupby("offence2")["count"].sum()
lev2_2020 =  df[(df["date"] >= "2020-01-01") & (df["date"] <= "2020-12-31")].groupby("offence2")["count"].sum()
lev2_all_years = df.groupby("offence2")["count"].sum()

In [43]:
lev2_2014["YEAR"]="2014"
lev2_2014 = pd.DataFrame(data=lev2_2014)
lev2_2014=lev2_2014.T
lev2_2014.set_index("YEAR")

offence2,ACTS INTENDED TO CAUSE INJURY,FRAUD DECEPTION AND RELATED OFFENCES,HOMICIDE AND RELATED OFFENCES,OTHER OFFENCES AGAINST THE PERSON,PROPERTY DAMAGE AND ENVIRONMENTAL,ROBBERY AND RELATED OFFENCES,SERIOUS CRIMINAL TRESPASS,SEXUAL ASSAULT AND RELATED OFFENCES,THEFT AND RELATED OFFENCES
YEAR,,,,,,,,,
2014,9049.0,1342.0,40.0,1821.0,11429.0,366.0,7400.0,1060.0,19722.0


In [44]:
lev2_2015["YEAR"]="2015"
lev2_2015 = pd.DataFrame(data=lev2_2015)
lev2_2015=lev2_2015.T
lev2_2015.set_index("YEAR")

offence2,ACTS INTENDED TO CAUSE INJURY,FRAUD DECEPTION AND RELATED OFFENCES,HOMICIDE AND RELATED OFFENCES,OTHER OFFENCES AGAINST THE PERSON,PROPERTY DAMAGE AND ENVIRONMENTAL,ROBBERY AND RELATED OFFENCES,SERIOUS CRIMINAL TRESPASS,SEXUAL ASSAULT AND RELATED OFFENCES,THEFT AND RELATED OFFENCES
YEAR,,,,,,,,,
2015,17993.0,2919.0,57.0,3606.0,23073.0,642.0,15570.0,2022.0,41416.0


In [45]:
lev2_2016["YEAR"]="2016"
lev2_2016 = pd.DataFrame(data=lev2_2016)
lev2_2016=lev2_2016.T
lev2_2016.set_index("YEAR")

offence2,ACTS INTENDED TO CAUSE INJURY,FRAUD DECEPTION AND RELATED OFFENCES,HOMICIDE AND RELATED OFFENCES,OTHER OFFENCES AGAINST THE PERSON,PROPERTY DAMAGE AND ENVIRONMENTAL,ROBBERY AND RELATED OFFENCES,SERIOUS CRIMINAL TRESPASS,SEXUAL ASSAULT AND RELATED OFFENCES,THEFT AND RELATED OFFENCES
YEAR,,,,,,,,,
2016,17519.0,2836.0,57.0,3026.0,22450.0,619.0,16901.0,1903.0,44253.0


In [47]:
lev2_2017["YEAR"]="2017"
lev2_2017 = pd.DataFrame(data=lev2_2017)
lev2_2017=lev2_2017.T
lev2_2017.set_index("YEAR")

offence2,ACTS INTENDED TO CAUSE INJURY,FRAUD DECEPTION AND RELATED OFFENCES,HOMICIDE AND RELATED OFFENCES,OTHER OFFENCES AGAINST THE PERSON,PROPERTY DAMAGE AND ENVIRONMENTAL,ROBBERY AND RELATED OFFENCES,SERIOUS CRIMINAL TRESPASS,SEXUAL ASSAULT AND RELATED OFFENCES,THEFT AND RELATED OFFENCES
YEAR,,,,,,,,,
2017,16665.0,2935.0,68.0,2943.0,21307.0,509.0,15016.0,2050.0,41729.0


In [46]:
lev2_2018["YEAR"]="2018"
lev2_2018 = pd.DataFrame(data=lev2_2018)
lev2_2018=lev2_2018.T
lev2_2018.set_index("YEAR")

offence2,ACTS INTENDED TO CAUSE INJURY,FRAUD DECEPTION AND RELATED OFFENCES,HOMICIDE AND RELATED OFFENCES,OTHER OFFENCES AGAINST THE PERSON,PROPERTY DAMAGE AND ENVIRONMENTAL,ROBBERY AND RELATED OFFENCES,SERIOUS CRIMINAL TRESPASS,SEXUAL ASSAULT AND RELATED OFFENCES,THEFT AND RELATED OFFENCES
YEAR,,,,,,,,,
2018,17217.0,3579.0,48.0,2332.0,21310.0,562.0,15559.0,1991.0,47503.0


In [48]:
lev2_2019["YEAR"]="2019"
lev2_2019 = pd.DataFrame(data=lev2_2019)
lev2_2019=lev2_2019.T
lev2_2019.set_index("YEAR")

offence2,ACTS INTENDED TO CAUSE INJURY,FRAUD DECEPTION AND RELATED OFFENCES,HOMICIDE AND RELATED OFFENCES,OTHER OFFENCES AGAINST THE PERSON,PROPERTY DAMAGE AND ENVIRONMENTAL,ROBBERY AND RELATED OFFENCES,SERIOUS CRIMINAL TRESPASS,SEXUAL ASSAULT AND RELATED OFFENCES,THEFT AND RELATED OFFENCES
YEAR,,,,,,,,,
2019,16471.0,3910.0,38.0,1801.0,21847.0,658.0,16432.0,1942.0,54743.0


In [49]:
lev2_2020["YEAR"]="2020"
lev2_2020 = pd.DataFrame(data=lev2_2020)
lev2_2020=lev2_2020.T
lev2_2020.set_index("YEAR")

offence2,ACTS INTENDED TO CAUSE INJURY,FRAUD DECEPTION AND RELATED OFFENCES,HOMICIDE AND RELATED OFFENCES,OTHER OFFENCES AGAINST THE PERSON,PROPERTY DAMAGE AND ENVIRONMENTAL,ROBBERY AND RELATED OFFENCES,SERIOUS CRIMINAL TRESPASS,SEXUAL ASSAULT AND RELATED OFFENCES,THEFT AND RELATED OFFENCES
YEAR,,,,,,,,,
2020,8539.0,1400.0,22.0,716.0,10191.0,285.0,7009.0,969.0,24387.0


In [50]:
lev2_all_years["YEAR"]="ALL-YEARS"
lev2_all_years = pd.DataFrame(data=lev2_all_years)
lev2_all_years=lev2_all_years.T
lev2_all_years.set_index("YEAR")

offence2,ACTS INTENDED TO CAUSE INJURY,FRAUD DECEPTION AND RELATED OFFENCES,HOMICIDE AND RELATED OFFENCES,OTHER OFFENCES AGAINST THE PERSON,PROPERTY DAMAGE AND ENVIRONMENTAL,ROBBERY AND RELATED OFFENCES,SERIOUS CRIMINAL TRESPASS,SEXUAL ASSAULT AND RELATED OFFENCES,THEFT AND RELATED OFFENCES
YEAR,,,,,,,,,
ALL-YEARS,103453.0,18921.0,330.0,16245.0,131607.0,3641.0,93887.0,11937.0,273753.0


In [51]:
frames = [lev2_2014, lev2_2015 , lev2_2016 , lev2_2017 , lev2_2018 , lev2_2019, lev2_2020, lev2_all_years]
lev2_combined = pd.concat(frames)

In [52]:
lev2_combined = lev2_combined.set_index("YEAR")

##### LEVEL 3 OFFENCE: BY YEAR

In [53]:
lev3_2014 =  df[(df["date"] >= "2014-01-01") & (df["date"] <= "2014-12-31")].groupby("offence3")["count"].sum()
lev3_2015 =  df[(df["date"] >= "2015-01-01") & (df["date"] <= "2015-12-31")].groupby("offence3")["count"].sum()
lev3_2016 =  df[(df["date"] >= "2016-01-01") & (df["date"] <= "2016-12-31")].groupby("offence3")["count"].sum()
lev3_2017 =  df[(df["date"] >= "2017-01-01") & (df["date"] <= "2017-12-31")].groupby("offence3")["count"].sum()
lev3_2018 =  df[(df["date"] >= "2018-01-01") & (df["date"] <= "2018-12-31")].groupby("offence3")["count"].sum()
lev3_2019 =  df[(df["date"] >= "2019-01-01") & (df["date"] <= "2019-12-31")].groupby("offence3")["count"].sum()
lev3_2020 =  df[(df["date"] >= "2020-01-01") & (df["date"] <= "2020-12-31")].groupby("offence3")["count"].sum()
lev3_all_years = df.groupby("offence3")["count"].sum()

In [54]:
lev3_2014["YEAR"]="2014"
lev3_2014 = pd.DataFrame(data=lev3_2014)
lev3_2014=lev3_2014.T
lev3_2014.set_index("YEAR")

offence3,"Abduction, harassment and other offences",Aggravated robbery,Aggravated sexual assault,Assault police,Blackmail and extortion,Common Assault,Dangerous or negligent acts,Graffiti,Murder,Non-aggravated robbery,...,Property damage by fire or explosion,Receive or handle proceeds of crime,SCT - Non Residence,SCT - Residence,Serious Assault not resulting in injury,Serious Assault resulting in injury,Theft from motor vehicle,Theft from shop,Theft/Illegal Use of MV,Threatening behaviour
YEAR,,,,,,,,,,,,,,,,,,,,,
2014,722.0,263.0,651.0,364.0,31.0,3457.0,278.0,1283.0,7.0,72.0,...,888.0,612.0,2307.0,3833.0,4300.0,567.0,4888.0,3218.0,1585.0,821.0


In [56]:
lev3_2015["YEAR"]="2015"
lev3_2015 = pd.DataFrame(data=lev3_2015)
lev3_2015=lev3_2015.T
lev3_2015.set_index("YEAR")

offence3,"Abduction, harassment and other offences",Aggravated robbery,Aggravated sexual assault,Assault police,Blackmail and extortion,Common Assault,Dangerous or negligent acts,Graffiti,Murder,Non-aggravated robbery,...,Property damage by fire or explosion,Receive or handle proceeds of crime,SCT - Non Residence,SCT - Residence,Serious Assault not resulting in injury,Serious Assault resulting in injury,Theft from motor vehicle,Theft from shop,Theft/Illegal Use of MV,Threatening behaviour
YEAR,,,,,,,,,,,,,,,,,,,,,
2015,1426.0,449.0,1219.0,763.0,70.0,6925.0,716.0,2487.0,20.0,123.0,...,1776.0,1314.0,4734.0,7971.0,8638.0,1003.0,9737.0,7127.0,3239.0,1464.0


In [55]:
lev3_2016["YEAR"]="2016"
lev3_2016 = pd.DataFrame(data=lev3_2016)
lev3_2016=lev3_2016.T
lev3_2016.set_index("YEAR")

offence3,"Abduction, harassment and other offences",Aggravated robbery,Aggravated sexual assault,Assault police,Blackmail and extortion,Common Assault,Dangerous or negligent acts,Graffiti,Murder,Non-aggravated robbery,...,Property damage by fire or explosion,Receive or handle proceeds of crime,SCT - Non Residence,SCT - Residence,Serious Assault not resulting in injury,Serious Assault resulting in injury,Theft from motor vehicle,Theft from shop,Theft/Illegal Use of MV,Threatening behaviour
YEAR,,,,,,,,,,,,,,,,,,,,,
2016,1113.0,438.0,1199.0,756.0,68.0,6744.0,655.0,2078.0,21.0,113.0,...,1518.0,1458.0,5034.0,8572.0,8490.0,904.0,9659.0,8201.0,3551.0,1258.0


In [57]:
lev3_2017["YEAR"]="2017"
lev3_2017 = pd.DataFrame(data=lev3_2017)
lev3_2017=lev3_2017.T
lev3_2017.set_index("YEAR")

offence3,"Abduction, harassment and other offences",Aggravated robbery,Aggravated sexual assault,Assault police,Blackmail and extortion,Common Assault,Dangerous or negligent acts,Graffiti,Murder,Non-aggravated robbery,...,Property damage by fire or explosion,Receive or handle proceeds of crime,SCT - Non Residence,SCT - Residence,Serious Assault not resulting in injury,Serious Assault resulting in injury,Theft from motor vehicle,Theft from shop,Theft/Illegal Use of MV,Threatening behaviour
YEAR,,,,,,,,,,,,,,,,,,,,,
2017,1259.0,345.0,1218.0,717.0,49.0,6731.0,726.0,2033.0,23.0,115.0,...,1635.0,1453.0,4448.0,7372.0,7850.0,869.0,8288.0,8440.0,3037.0,958.0


In [58]:
lev3_2018["YEAR"]="2018"
lev3_2018 = pd.DataFrame(data=lev3_2018)
lev3_2018=lev3_2018.T
lev3_2018.set_index("YEAR")

offence3,"Abduction, harassment and other offences",Aggravated robbery,Aggravated sexual assault,Assault police,Blackmail and extortion,Common Assault,Dangerous or negligent acts,Graffiti,Murder,Non-aggravated robbery,...,Property damage by fire or explosion,Receive or handle proceeds of crime,SCT - Non Residence,SCT - Residence,Serious Assault not resulting in injury,Serious Assault resulting in injury,Theft from motor vehicle,Theft from shop,Theft/Illegal Use of MV,Threatening behaviour
YEAR,,,,,,,,,,,,,,,,,,,,,
2018,723.0,384.0,1228.0,823.0,59.0,7004.0,757.0,1784.0,15.0,119.0,...,1588.0,1757.0,4155.0,7862.0,7736.0,1109.0,9835.0,9766.0,3361.0,852.0


In [59]:
lev3_2019["YEAR"]="2019"
lev3_2019 = pd.DataFrame(data=lev3_2019)
lev3_2019=lev3_2019.T
lev3_2019.set_index("YEAR")

offence3,"Abduction, harassment and other offences",Aggravated robbery,Aggravated sexual assault,Assault police,Blackmail and extortion,Common Assault,Dangerous or negligent acts,Graffiti,Murder,Non-aggravated robbery,...,Property damage by fire or explosion,Receive or handle proceeds of crime,SCT - Non Residence,SCT - Residence,Serious Assault not resulting in injury,Serious Assault resulting in injury,Theft from motor vehicle,Theft from shop,Theft/Illegal Use of MV,Threatening behaviour
YEAR,,,,,,,,,,,,,,,,,,,,,
2019,471.0,482.0,1112.0,619.0,61.0,5573.0,597.0,1704.0,13.0,115.0,...,1326.0,1659.0,4606.0,8469.0,6875.0,2920.0,11164.0,11649.0,4164.0,733.0


In [60]:
lev3_2020["YEAR"]="2020"
lev3_2020 = pd.DataFrame(data=lev3_2020)
lev3_2020=lev3_2020.T
lev3_2020.set_index("YEAR")

offence3,"Abduction, harassment and other offences",Aggravated robbery,Aggravated sexual assault,Assault police,Blackmail and extortion,Common Assault,Dangerous or negligent acts,Graffiti,Murder,Non-aggravated robbery,...,Property damage by fire or explosion,Receive or handle proceeds of crime,SCT - Non Residence,SCT - Residence,Serious Assault not resulting in injury,Serious Assault resulting in injury,Theft from motor vehicle,Theft from shop,Theft/Illegal Use of MV,Threatening behaviour
YEAR,,,,,,,,,,,,,,,,,,,,,
2020,205.0,202.0,559.0,254.0,25.0,1108.0,152.0,788.0,4.0,58.0,...,503.0,820.0,2342.0,2961.0,5453.0,1407.0,4782.0,5432.0,1939.0,359.0


In [61]:
lev3_all_years["YEAR"]="ALL-YEARS"
lev3_all_years = pd.DataFrame(data=lev3_all_years)
lev3_all_years=lev3_all_years.T
lev3_all_years.set_index("YEAR")

offence3,"Abduction, harassment and other offences",Aggravated robbery,Aggravated sexual assault,Assault police,Blackmail and extortion,Common Assault,Dangerous or negligent acts,Graffiti,Murder,Non-aggravated robbery,...,Property damage by fire or explosion,Receive or handle proceeds of crime,SCT - Non Residence,SCT - Residence,Serious Assault not resulting in injury,Serious Assault resulting in injury,Theft from motor vehicle,Theft from shop,Theft/Illegal Use of MV,Threatening behaviour
YEAR,,,,,,,,,,,,,,,,,,,,,
ALL-YEARS,5919.0,2563.0,7186.0,4296.0,363.0,37542.0,3881.0,12157.0,103.0,715.0,...,9234.0,9073.0,27626.0,47040.0,49342.0,8779.0,58353.0,53833.0,20876.0,6445.0


In [62]:
frames = [lev3_2014, lev3_2015 , lev3_2016 , lev3_2017 , lev3_2018 , lev3_2019, lev3_2020, lev3_all_years]
lev3_combined = pd.concat(frames)

In [63]:
lev3_combined = lev3_combined.set_index("YEAR")

##### TOP 10 LOCALITIES FOR ALL CRIMES

In [64]:
locality_df = df.groupby("locality").sum()["count"]
locality_df = pd.merge(locality_df,sa_geo_info_cleaned, on="locality", how="left")
locality_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1598 entries, 0 to 1597
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   locality  1598 non-null   object 
 1   count     1598 non-null   float64
 2   lat       1538 non-null   float64
 3   long      1538 non-null   float64
dtypes: float64(3), object(1)
memory usage: 62.4+ KB


In [65]:
# checking localities with most offence counts

top10_localities = locality_df.sort_values("count", ascending=False).reset_index(drop=True).head(10)
top10_localities.index = top10_localities.index + 1
top10_localities

,locality,count,lat,long
1,ADELAIDE,36463.0,-34.937459,138.608637
2,NOT DISCLOSED,11937.0,NaN,NaN
3,MORPHETT VALE,10044.0,-35.112119,138.565224
4,MURRAY BRIDGE,9626.0,-35.109083,139.273641
5,MOUNT GAMBIER,8924.0,-37.839841,140.764545
6,PORT AUGUSTA,8891.0,-32.735760,137.703073
7,SALISBURY,8510.0,-34.761553,138.618840
8,PORT LINCOLN,7687.0,-35.373243,136.859338
9,ELIZABETH,7470.0,-34.726229,138.666615
10,OAKLANDS PARK,7030.0,-35.003794,138.535434


### SAVING ALL TABLES TO SQLITE DATABASE

In [66]:
database = "../sa_crime_new.sqlite"
conn = sqlite3.connect(database)

In [67]:
df.to_sql(name="main_df", con=conn, if_exists="fail")
lev1_combined.to_sql(name="lev1_combined", con=conn, if_exists="fail")
lev2_combined.to_sql(name="lev2_combined", con=conn, if_exists="fail")
lev3_combined.to_sql(name="lev3_combined", con=conn, if_exists="fail")
top10_localities.to_sql(name="top10_localities", con=conn, if_exists="fail")


/Users/shrutihindocha/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/pandas/core/generic.py:2882: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,
